In [3]:
# 필요한 패키지 참조

from pandas import read_excel, DataFrame, pivot_table, to_datetime, concat
import numpy as np
import sys
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime,timedelta 
import pandas as pd

In [85]:
# 데이터 가져오기

origin = read_excel('C:/Users/J/Desktop/미세먼지data/1. 미세먼지측정(환경부_21~23_AirKorea)/2023/2023년_8월.xlsx')
origin.head()

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2023080101,0.0023,0.40,0.0146,0.0245,NaN,18.0,서울 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2023080102,0.0023,0.35,0.0223,0.0181,18.0,15.0,서울 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2023080103,0.0024,0.34,0.0184,0.0193,NaN,17.0,서울 중구 덕수궁길 15
3,서울 중구,도시대기,111121,중구,2023080104,0.0023,0.33,0.0190,0.0173,17.0,15.0,서울 중구 덕수궁길 15
4,서울 중구,도시대기,111121,중구,2023080105,0.0019,0.30,0.0216,0.0139,17.0,16.0,서울 중구 덕수궁길 15


In [19]:
# 데이터 확인

origin.dtypes

지역        object
망         object
측정소코드      int64
측정소명      object
측정일시       int64
SO2      float64
CO       float64
O3       float64
NO2      float64
PM10     float64
PM25     float64
주소        object
dtype: object

In [20]:
origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470208 entries, 0 to 470207
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   지역      470208 non-null  object 
 1   망       470208 non-null  object 
 2   측정소코드   470208 non-null  int64  
 3   측정소명    470208 non-null  object 
 4   측정일시    470208 non-null  int64  
 5   SO2     447803 non-null  float64
 6   CO      444416 non-null  float64
 7   O3      447787 non-null  float64
 8   NO2     448772 non-null  float64
 9   PM10    440578 non-null  float64
 10  PM25    434079 non-null  float64
 11  주소      470208 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 43.0+ MB


In [86]:
# 서울, 도시대기, 측정일시, PM2.5만 추출 

df = origin[origin['지역'].str.contains('서울')]
df2 = df.query('망 == "도시대기"')
df3 = df2[['지역','측정일시','PM25']]
df3['측정일시']=df3['측정일시'].astype('str')
df3
# df4 = df3.pivot_table(index=['지역'],columns='측정일시', values='PM25')

C:\Users\J\AppData\Local\Temp\ipykernel_17452\575013061.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['측정일시']=df3['측정일시'].astype('str')


,지역,측정일시,PM25
0,서울 중구,2023080101,18.0
1,서울 중구,2023080102,15.0
2,서울 중구,2023080103,17.0
3,서울 중구,2023080104,15.0
4,서울 중구,2023080105,16.0
...,...,...,...
29011,서울 노원구,2023083120,9.0
29012,서울 노원구,2023083121,11.0
29013,서울 노원구,2023083122,8.0
29014,서울 노원구,2023083123,14.0


In [87]:
# 날짜 변환 함수 정의
def convert_date_string(date_str):
    # 마지막 두 글자가 '24'일 경우에만 처리
    if date_str[-2:] == '24':
        # 마지막 두 글자를 시간으로 인식하여 '00'시로 설정
        date_str = date_str[:-2] + '00'
        # 문자열을 날짜와 시간 객체로 변환
        date_time_obj = datetime.strptime(date_str, '%Y%m%d%H')
        # 날짜를 1일 늘리기
        date_time_obj += timedelta(days=1)
    else:
        # 마지막 두 글자가 '24'가 아닐 경우에는 그대로 변환
        date_time_obj = datetime.strptime(date_str, '%Y%m%d%H')
    return date_time_obj

# 데이터프레임의 'date_column' 컬럼에 있는 값을 변환
df3['측정일시'] = df3['측정일시'].apply(convert_date_string)

# 결과 출력


C:\Users\J\AppData\Local\Temp\ipykernel_17452\928624868.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['측정일시'] = df3['측정일시'].apply(convert_date_string)


In [88]:
df4 = df3.copy()
df4


,지역,측정일시,PM25
0,서울 중구,2023-08-01 01:00:00,18.0
1,서울 중구,2023-08-01 02:00:00,15.0
2,서울 중구,2023-08-01 03:00:00,17.0
3,서울 중구,2023-08-01 04:00:00,15.0
4,서울 중구,2023-08-01 05:00:00,16.0
...,...,...,...
29011,서울 노원구,2023-08-31 20:00:00,9.0
29012,서울 노원구,2023-08-31 21:00:00,11.0
29013,서울 노원구,2023-08-31 22:00:00,8.0
29014,서울 노원구,2023-08-31 23:00:00,14.0


In [89]:
# 일별 평균 구하는 파생변수 생성

# 컬럼 나누기(년월일/시간)

df4['년'] = df4['측정일시'].dt.year
df4['월'] = df4['측정일시'].dt.month
df4['일'] = df4['측정일시'].dt.day
df4['시간'] = df4['측정일시'].dt.hour
df4['년']=df4['년'].astype('str')
df4['월']=df4['월'].astype('str')
df4['일']=df4['일'].astype('str')
df4.dtypes
df4['년월일'] = df4['년'].map(str) + "-" + df4['월'].map(str) + "-" + df4['일'].map(str)
df4['년월일']=to_datetime(df4['년월일'])
df5 = df4[['지역','년월일','시간','PM25']]
df6 = df5.pivot_table(index=['지역','년월일'],columns='시간',values='PM25')
df6['일별평균']=df6.iloc[:,:24].mean(axis=1)
df7 = df6.reset_index('지역')
df7.to_excel('외부_2308.xlsx')




In [91]:
origin1 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2208.xlsx')
origin2 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2209.xlsx')
origin3 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2210.xlsx')
origin4 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2211.xlsx')
origin5 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2212.xlsx')
origin6 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2301.xlsx')
origin7 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2302.xlsx')
origin8 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2303.xlsx')
origin9 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2304.xlsx')
origin10 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2305.xlsx')
origin11 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2306.xlsx')
origin12 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2307.xlsx')
origin13 = read_excel('C:/Users/J/Desktop/미세먼지data/필수데이터_전처리코드/외부_2308.xlsx')


In [93]:
tdf = concat([origin1,origin2,origin3,origin4,origin5,origin6,origin7,origin8,origin9,origin10,origin11,origin12,origin13])
tdf.to_excel('외부미세먼지최종_(2208~2308).xlsx')